In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from getpass import getpass

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth, association_rules

import keyring
import time

import requests

import requests
from urllib.parse import urljoin
import bs4
from sklearn.metrics import pairwise_distances as dist
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from scipy.spatial.distance import mahalanobis
from scipy.linalg import inv
from sklearn.preprocessing import StandardScaler

In [ ]:
client_id = getpass()

In [ ]:
password = getpass()

In [ ]:
# For SPOTIPY api
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
# For Spotify API

# Spotify URL for obtaining an access token
auth_url = 'https://accounts.spotify.com/api/token'

# Make a request to get an access token
auth_response = requests.post(auth_url, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

# Convert the response to JSON
auth_response_data = auth_response.json()

# Save the access token
access_token = auth_response_data['access_token']

In [ ]:
df_coffee_playlist_songs = pd.read_csv('df_coffee_final')
df_coffee_playlist_songs = df_coffee_playlist_songs[
    ~df_coffee_playlist_songs['playlist_name'].str.contains(
        'korean', case=False, na=False
    )
]
df_coffee_playlist_songs['song_name'] = (
    df_coffee_playlist_songs['song_name'].str.strip()
)
df_coffee_playlist_songs['playlist_id'] = (
    df_coffee_playlist_songs['playlist_id'].astype('str')
)
filtered_df = df_coffee_playlist_songs.groupby('song_name').filter(
    lambda x: x['song_id'].nunique() > 1
)
sorted_filtered_df = filtered_df.sort_values(by='song_name')
filtered_df_nokoja = sorted_filtered_df[
    ~sorted_filtered_df['song_name'].str.contains(
        '[\u3040-\u30FF\u4E00-\u9FAF\uAC00-\uD7AF]', regex=True, na=False
    )
]
filtered_df_nokoja.head(3)

,playlist_id,playlist_name,song_name,artist_name,song_id,artist_id
59570,2qw26tftXKmG0BqAhaqMaC,coffee shop vibes,"""Slut!"" (Taylor's Version) (From The Vault)",Taylor Swift,0CD7DzeCsuPJygddqlUVYa,06HL4z0CvFAxyc27GXpf02
56286,4SYNPEghk6LHX2WNxiA4PL,coffee shop soundtrack,"""Slut!"" (Taylor's Version) (From The Vault)",Taylor Swift,0CD7DzeCsuPJygddqlUVYa,06HL4z0CvFAxyc27GXpf02
113891,0vGBxvd7y0hWJSt3vKx5Fx,cafe playlist,"""Slut!"" (Taylor's Version) (From The Vault)",Taylor Swift,71BqAINEnezjQfxE4VuJfq,06HL4z0CvFAxyc27GXpf02


In [ ]:
# Convert to basket format with playlist_id as index and songs as columns
basket = (filtered_df_nokoja
          .groupby(['playlist_id', 'song_id'])['song_id']
          .count().unstack().reset_index().fillna(0)
          .set_index('playlist_id'))

basket

song_id,00009c9uMfez9Zqj1kvJkL,000DcCEzHlmBNoVJ6CbMnw,0027zf5idrflGmhXsvdVJr,003eoIwxETJujVWmNFMoZy,003qa9jZ2byfApC0hBnr0G,003vvx7Niy0yvhvHt4a68B,005ZaBYjutYvK4VLYG6QwA,006AQyEI56rMRXZ7hO4YfB,006bxNsqWbbtiyLWyU9hoz,0087fDVeNhuyMtST91RMjv,...,7zo53YenWv2I2cSRxkM24D,7zoImkPg0DGeyhHroWSDh5,7zpepCjyubEIjpN70H3aYc,7zppfPzURuoTtlrZYMvHFM,7zrPswSV1yxrill5OyCuaU,7zu4F2AtCvns6DINJCJ5AM,7zuS45SBDGxsjccv09A5jt,7zv1YHGW4P9HUkdDLzNlFX,7zvwxa2s4zIX7y49plhrmo,7zzLFsWaWsPOfqoac4VqPZ
playlist_id,,,,,,,,,,,,,,,,,,,,,
03HXXpk7MjL0hEm9IsIbRv,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
03tFFBbrsZKwkpfmYQ7cZA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06TY2MQqB4GKSORxKmr8ws,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
08hrBmZsseF6vfUwdLkNiU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
09pKPWU0uHhcIgArAOVGse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7wH4GLpCrO1IdBxmbWKgNF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7x5pKM8zd5RGrkT9pq7pGd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7z8upKqxqY1c77UJyGgXSV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 0:
        return 1

basket_sets = basket.applymap(encode_units)

## FP-Growth

In [ ]:
fp_growth_fim = fpgrowth(basket_sets, min_support=0.011, use_colnames=True)
rules = association_rules(fp_growth_fim,
                          metric='confidence',
                          min_threshold=0.1)

rules

/home/msds2024/gservanez/.local/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(3X9c4tBzSdGhlO4Fx3WYgW),(6wdCelHrPh7UfliNjwRTUv),0.018476,0.013857,0.011547,0.625000,45.104167,0.011291,2.629715,0.996235
1,(6wdCelHrPh7UfliNjwRTUv),(3X9c4tBzSdGhlO4Fx3WYgW),0.013857,0.018476,0.011547,0.833333,45.104167,0.011291,5.889145,0.991569
2,(754kgU5rWscRTfvlsuEwFp),(0ofHAoxe9vBkTCp2UQIavz),0.066975,0.099307,0.023095,0.344828,3.472334,0.016444,1.374742,0.763119
3,(0ofHAoxe9vBkTCp2UQIavz),(754kgU5rWscRTfvlsuEwFp),0.099307,0.066975,0.023095,0.232558,3.472334,0.016444,1.215760,0.790513
4,(3AJwUDP919kvQ9QcozQPxg),(754kgU5rWscRTfvlsuEwFp),0.071594,0.066975,0.013857,0.193548,2.889878,0.009062,1.156952,0.704395
...,...,...,...,...,...,...,...,...,...,...
10875,"(6UelLqGlWMcVH1E5c4H7lY, 6b2RcmUt1g9N9mQ3CbjX2Y)",(3HVWdVOQ0ZA45FuZGSfvns),0.011547,0.023095,0.011547,1.000000,43.300000,0.011281,inf,0.988318
10876,"(3HVWdVOQ0ZA45FuZGSfvns, 6b2RcmUt1g9N9mQ3CbjX2Y)",(6UelLqGlWMcVH1E5c4H7lY),0.011547,0.050808,0.011547,1.000000,19.681818,0.010961,inf,0.960280
10877,(6UelLqGlWMcVH1E5c4H7lY),"(3HVWdVOQ0ZA45FuZGSfvns, 6b2RcmUt1g9N9mQ3CbjX2Y)",0.050808,0.011547,0.011547,0.227273,19.681818,0.010961,1.279174,1.000000
10878,(3HVWdVOQ0ZA45FuZGSfvns),"(6UelLqGlWMcVH1E5c4H7lY, 6b2RcmUt1g9N9mQ3CbjX2Y)",0.023095,0.011547,0.011547,0.500000,43.300000,0.011281,1.976905,1.000000


In [ ]:
rules['set_size'] = rules['antecedents'].apply(lambda x: len(x))
rules.sort_values(by='set_size',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,set_size
2070,"(0BCPKOYdS2jbQ8iyB56Zns, 5kRPPEWFJIMox5qIkQkiz...",(3AJwUDP919kvQ9QcozQPxg),0.011547,0.071594,0.011547,1.000000,13.967742,0.010721,inf,0.939252,5
2071,"(3AJwUDP919kvQ9QcozQPxg, 5kRPPEWFJIMox5qIkQkiz...",(0BCPKOYdS2jbQ8iyB56Zns),0.013857,0.032333,0.011547,0.833333,25.773810,0.011099,5.806005,0.974707,5
2066,"(0BCPKOYdS2jbQ8iyB56Zns, 3AJwUDP919kvQ9QcozQPx...",(57RA3JGafJm5zRtKJiKPIm),0.011547,0.048499,0.011547,1.000000,20.619048,0.010987,inf,0.962617,5
2068,"(0BCPKOYdS2jbQ8iyB56Zns, 3AJwUDP919kvQ9QcozQPx...",(4U45aEWtQhrm8A5mxPaFZ7),0.011547,0.050808,0.011547,1.000000,19.681818,0.010961,inf,0.960280,5
2069,"(0BCPKOYdS2jbQ8iyB56Zns, 3AJwUDP919kvQ9QcozQPx...",(5kRPPEWFJIMox5qIkQkiz5),0.011547,0.050808,0.011547,1.000000,19.681818,0.010961,inf,0.960280,5
...,...,...,...,...,...,...,...,...,...,...,...
4583,(4Jz4bjXeiF2SXVj9P4YfY5),(4wTChU0tU44TAMC0xcQizi),0.013857,0.016166,0.013857,1.000000,61.857143,0.013633,inf,0.997658,1
4582,(4wTChU0tU44TAMC0xcQizi),(4Jz4bjXeiF2SXVj9P4YfY5),0.016166,0.013857,0.013857,0.857143,61.857143,0.013633,6.903002,1.000000,1
4581,(0KAiuUOrLTIkzkpfpn9jb9),(4HlFJV71xXKIGcU3kRyttv),0.013857,0.032333,0.011547,0.833333,25.773810,0.011099,5.806005,0.974707,1
4580,(4HlFJV71xXKIGcU3kRyttv),(0KAiuUOrLTIkzkpfpn9jb9),0.032333,0.013857,0.011547,0.357143,25.773810,0.011099,1.534001,0.993317,1


### Recommendation using Input Songs

In [ ]:
def preprocess_rules(rules):
    """
    Preprocess rules to facilitate easy lookup.
    Converts frozensets to regular sets for easier manipulation.
    """
    # Convert frozensets to sets for easier manipulation
    rules['antecedents'] = rules['antecedents'].apply(set)
    rules['consequents'] = rules['consequents'].apply(set)
    return rules

def recommend_songs(input_songs, rules, num_recommendations=100):
    """
    Generate song recommendations based on input songs and association rules.
    Enhancements for diversity, feedback integration, and dynamic
    updates are suggested.
    """
    input_set = set(input_songs)

    # Add the size of antecedents
    rules['set_size'] = rules['antecedents'].apply(lambda x: len(x))

    # Initially filter rules where antecedents are a subset of the input songs
    relevant_rules = rules[
    rules['antecedents'].apply(lambda x: x.issubset(input_set))
    ]

    # Sort by descending set_size, confidence, and lift
    relevant_rules = relevant_rules.sort_values(by=['set_size',
                                                    'confidence',
                                                    'lift'],
                                                ascending=[False,
                                                           False,
                                                           False])

    recommendations = []
    seen = set()

    # Iterate through sorted relevant rules
    for _, row in relevant_rules.iterrows():
        for song in row['consequents']:
            if (song not in input_set and song not in seen and
                len(recommendations) < num_recommendations):
                recommendations.append(song)
                seen.add(song)

    return recommendations

In [ ]:
#Implementation

input_songs = ['0ofHAoxe9vBkTCp2UQIavz',
               '3X9c4tBzSdGhlO4Fx3WYgW',
               '3AJwUDP919kvQ9QcozQPxg']


In [ ]:
# Fetch song and artist names
track_details = sp.tracks(input_songs)
songs_with_artists = [
    (track['name'], ", ".join([artist['name'] for artist in track['artists']]))
    for track in track_details['tracks']
]

# Print song names with artist names
for name, artists in songs_with_artists:
    print(f"{name} by {artists}")

Dreams - 2004 Remaster by Fleetwood Mac
Lifetime by Ben&Ben
Yellow by Coldplay


In [ ]:
rules_processed = preprocess_rules(rules)
recommended_songs = recommend_songs(input_songs, rules_processed)
print("These are the recommended songs:\n", recommended_songs[:10])

These are the recommended songs:
 ['0BCPKOYdS2jbQ8iyB56Zns', '57RA3JGafJm5zRtKJiKPIm', '4LRPiXqCikLlN15c3yImP7', '23khhseCLQqVMCIT1WMAns', '2GiJYvgVaD2HtM8GqD9EgQ', '2QjOHCTQ1Jl3zawyYOpxh6', '7D0RhFcb3CrfPuTJ0obrod', '75JFxkI2RXiU7L9VXzMkle', '1qEmFfgcLObUfQm0j1W2CK', '754kgU5rWscRTfvlsuEwFp']


In [ ]:
# Fetch song and artist names
track_details = sp.tracks(recommended_songs[:10])
songs_with_artists = [
    (track["name"], ", ".join([artist["name"] for artist in track["artists"]]))
    for track in track_details["tracks"]
]

# Print song names with artist names
print("Here are the recommended songs including its artists:")
for name, artists in songs_with_artists:
    print(f"{name} by {artists}")

Here are the recommended songs including its artists:
Clocks by Coldplay
Are You Bored Yet? (feat. Clairo) by Wallows, Clairo
As It Was by Harry Styles
Magic by Coldplay
Electric Love by BØRNS
Sweater Weather by The Neighbourhood
Sparks by Coldplay
The Scientist by Coldplay
Late Night Talking by Harry Styles
Kiss Me by Sixpence None The Richer


In [ ]:
top_10_songs = recommended_songs[:10]
top_10_songs

['0BCPKOYdS2jbQ8iyB56Zns',
 '57RA3JGafJm5zRtKJiKPIm',
 '4LRPiXqCikLlN15c3yImP7',
 '23khhseCLQqVMCIT1WMAns',
 '2GiJYvgVaD2HtM8GqD9EgQ',
 '2QjOHCTQ1Jl3zawyYOpxh6',
 '7D0RhFcb3CrfPuTJ0obrod',
 '75JFxkI2RXiU7L9VXzMkle',
 '1qEmFfgcLObUfQm0j1W2CK',
 '754kgU5rWscRTfvlsuEwFp']

In [ ]:
def get_audio_features(list_songs):
    sleep_min = 1
    sleep_max = 3
    request_count = 0
    successful_requests = 0
    start_time = time.time()

    audio_features_list = []

    # Base URL for Spotify API
    base_url = 'https://api.spotify.com/v1/audio-features/'

    # Iterate over each song_id in the DataFrame
    for song_id in list_songs:
        successful_request = False
        while not successful_request:
            # Increment request count
            request_count += 1

            # Make a request for the audio features of the track
            response = requests.get(f'{base_url}{song_id}', headers=headers)

            # Check if the request was successful
            if response.status_code == 200:
                successful_request = True
                successful_requests += 1

                # Convert the response to JSON
                audio_features = response.json()

                # Append the fetched audio features to the list
                audio_features_list.append({
                    'song_id': song_id,  # Keep the song_id for merging
                    'danceability': audio_features.get('danceability'),
                    'energy': audio_features.get('energy'),
                    'loudness': audio_features.get('loudness'),
                    'speechiness': audio_features.get('speechiness'),
                    'acousticness': audio_features.get('acousticness'),
                    'instrumentalness': audio_features.get('instrumentalness'),
                    'liveness': audio_features.get('liveness'),
                    'valence': audio_features.get('valence'),
                    'tempo': audio_features.get('tempo'),
                    'duration_ms': audio_features.get('duration_ms')
                })

                # Pause briefly to avoid rate limiting
                time.sleep(np.random.uniform(sleep_min, sleep_max))

            else:
                # Handle rate limiting or other errors
                print(
                    f"Error for {song_id}: {response.status_code}. Retrying."
                    )
                time.sleep(np.random.uniform(sleep_min, sleep_max) * 1.1)

    return pd.DataFrame(audio_features_list)

In [ ]:
top_10_songs_df = get_audio_features(top_10_songs)
top_10_songs_df

,song_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0BCPKOYdS2jbQ8iyB56Zns,0.577,0.749,-7.215,0.0279,0.59900,0.011500,0.1830,0.255,130.970,307880
1,57RA3JGafJm5zRtKJiKPIm,0.682,0.683,-6.444,0.0287,0.15600,0.000023,0.2730,0.640,120.023,178000
2,4LRPiXqCikLlN15c3yImP7,0.520,0.731,-5.338,0.0557,0.34200,0.001010,0.3110,0.662,173.930,167303
3,23khhseCLQqVMCIT1WMAns,0.738,0.482,-8.917,0.0402,0.02660,0.621000,0.1060,0.352,93.457,285014
4,2GiJYvgVaD2HtM8GqD9EgQ,0.611,0.797,-7.627,0.0533,0.00543,0.001370,0.2600,0.518,120.041,218107
5,2QjOHCTQ1Jl3zawyYOpxh6,0.612,0.807,-2.810,0.0336,0.04950,0.017700,0.1010,0.398,124.053,240400
6,7D0RhFcb3CrfPuTJ0obrod,0.371,0.268,-10.506,0.0281,0.74800,0.051700,0.1040,0.165,102.617,227093
7,75JFxkI2RXiU7L9VXzMkle,0.557,0.442,-7.224,0.0243,0.73100,0.000015,0.1100,0.213,146.277,309600
8,1qEmFfgcLObUfQm0j1W2CK,0.714,0.728,-4.595,0.0468,0.29800,0.000000,0.1060,0.901,114.996,177955
9,754kgU5rWscRTfvlsuEwFp,0.526,0.749,-6.964,0.0283,0.02300,0.000002,0.0692,0.421,99.982,208680


In [ ]:
input_audio_features = get_audio_features(input_songs)
input_audio_features

,song_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0ofHAoxe9vBkTCp2UQIavz,0.828,0.492,-9.744,0.0276,0.06440,0.004280,0.128,0.789,120.151,257800
1,3X9c4tBzSdGhlO4Fx3WYgW,0.386,0.657,-5.125,0.0316,0.29200,0.000010,0.393,0.401,179.998,171527
2,3AJwUDP919kvQ9QcozQPxg,0.429,0.661,-7.227,0.0281,0.00239,0.000121,0.234,0.285,173.372,266773


In [ ]:
# Input audio features with song name and artist

input_audio_features_with_name = pd.merge(
    input_audio_features, filtered_df_nokoja[['song_id',
                                              'song_name',
                                              'artist_name']],
    on='song_id',
    how='left'
    )
input_audio_features_with_name.drop_duplicates(inplace=True)
input_audio_features_with_name

,song_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,song_name,artist_name
0,0ofHAoxe9vBkTCp2UQIavz,0.828,0.492,-9.744,0.0276,0.06440,0.004280,0.128,0.789,120.151,257800,Dreams - 2004 Remaster,Fleetwood Mac
43,3X9c4tBzSdGhlO4Fx3WYgW,0.386,0.657,-5.125,0.0316,0.29200,0.000010,0.393,0.401,179.998,171527,Lifetime,Ben&Ben
51,3AJwUDP919kvQ9QcozQPxg,0.429,0.661,-7.227,0.0281,0.00239,0.000121,0.234,0.285,173.372,266773,Yellow,Coldplay


In [ ]:
# Recommended songs' audio features with song name and artist
recommended_audio_features_with_name = pd.merge(
    top_10_songs_df, filtered_df_nokoja[['song_id',
                                         'song_name',
                                         'artist_name']],
    on='song_id',
    how='left'
    )
recommended_audio_features_with_name.drop_duplicates(inplace=True)
recommended_audio_features_with_name

,song_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,song_name,artist_name
0,0BCPKOYdS2jbQ8iyB56Zns,0.577,0.749,-7.215,0.0279,0.59900,0.011500,0.1830,0.255,130.970,307880,Clocks,Coldplay
14,57RA3JGafJm5zRtKJiKPIm,0.682,0.683,-6.444,0.0287,0.15600,0.000023,0.2730,0.640,120.023,178000,Are You Bored Yet? (feat. Clairo),Wallows
35,4LRPiXqCikLlN15c3yImP7,0.520,0.731,-5.338,0.0557,0.34200,0.001010,0.3110,0.662,173.930,167303,As It Was,Harry Styles
64,23khhseCLQqVMCIT1WMAns,0.738,0.482,-8.917,0.0402,0.02660,0.621000,0.1060,0.352,93.457,285014,Magic,Coldplay
76,2GiJYvgVaD2HtM8GqD9EgQ,0.611,0.797,-7.627,0.0533,0.00543,0.001370,0.2600,0.518,120.041,218107,Electric Love,BØRNS
91,2QjOHCTQ1Jl3zawyYOpxh6,0.612,0.807,-2.810,0.0336,0.04950,0.017700,0.1010,0.398,124.053,240400,Sweater Weather,The Neighbourhood
110,7D0RhFcb3CrfPuTJ0obrod,0.371,0.268,-10.506,0.0281,0.74800,0.051700,0.1040,0.165,102.617,227093,Sparks,Coldplay
132,75JFxkI2RXiU7L9VXzMkle,0.557,0.442,-7.224,0.0243,0.73100,0.000015,0.1100,0.213,146.277,309600,The Scientist,Coldplay
156,1qEmFfgcLObUfQm0j1W2CK,0.714,0.728,-4.595,0.0468,0.29800,0.000000,0.1060,0.901,114.996,177955,Late Night Talking,Harry Styles
181,754kgU5rWscRTfvlsuEwFp,0.526,0.749,-6.964,0.0283,0.02300,0.000002,0.0692,0.421,99.982,208680,Kiss Me,Sixpence None The Richer


### Validation using distance measures

In this section, the effectiveness and relevance of the songs recommended by our model will be assessed through various distance measures. These measures will quantify the similarity between the recommended songs and a set of exemplar songs (input songs) considered as a standard or benchmark.

In [ ]:
# Exemplar is the input songs
exemplar = input_audio_features_with_name.set_index(keys=['song_id']).drop(
    columns=['duration_ms', 'song_name', 'artist_name']
    )
exemplar

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
song_id,,,,,,,,,
0ofHAoxe9vBkTCp2UQIavz,0.828,0.492,-9.744,0.0276,0.06440,0.004280,0.128,0.789,120.151
3X9c4tBzSdGhlO4Fx3WYgW,0.386,0.657,-5.125,0.0316,0.29200,0.000010,0.393,0.401,179.998
3AJwUDP919kvQ9QcozQPxg,0.429,0.661,-7.227,0.0281,0.00239,0.000121,0.234,0.285,173.372


In [ ]:
# Recommended Songs
reco_songs = recommended_audio_features_with_name.set_index(
    keys=['song_id']
    ).drop(
        columns=['duration_ms', 'song_name', 'artist_name']
        )
reco_songs

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
song_id,,,,,,,,,
0BCPKOYdS2jbQ8iyB56Zns,0.577,0.749,-7.215,0.0279,0.59900,0.011500,0.1830,0.255,130.970
57RA3JGafJm5zRtKJiKPIm,0.682,0.683,-6.444,0.0287,0.15600,0.000023,0.2730,0.640,120.023
4LRPiXqCikLlN15c3yImP7,0.520,0.731,-5.338,0.0557,0.34200,0.001010,0.3110,0.662,173.930
23khhseCLQqVMCIT1WMAns,0.738,0.482,-8.917,0.0402,0.02660,0.621000,0.1060,0.352,93.457
2GiJYvgVaD2HtM8GqD9EgQ,0.611,0.797,-7.627,0.0533,0.00543,0.001370,0.2600,0.518,120.041
2QjOHCTQ1Jl3zawyYOpxh6,0.612,0.807,-2.810,0.0336,0.04950,0.017700,0.1010,0.398,124.053
7D0RhFcb3CrfPuTJ0obrod,0.371,0.268,-10.506,0.0281,0.74800,0.051700,0.1040,0.165,102.617
75JFxkI2RXiU7L9VXzMkle,0.557,0.442,-7.224,0.0243,0.73100,0.000015,0.1100,0.213,146.277
1qEmFfgcLObUfQm0j1W2CK,0.714,0.728,-4.595,0.0468,0.29800,0.000000,0.1060,0.901,114.996


In [ ]:
# Normalization
scaler = MinMaxScaler()
df_tracks_reco_scaled = scaler.fit_transform(reco_songs)
exemplar_scaled = scaler.transform(exemplar)

In [ ]:
# Compute Pairwise Distances for 3 types of Similarity Measure
n = len(reco_songs)
l1_dist = dist(df_tracks_reco_scaled, exemplar_scaled, metric='l1')

l2_dist = dist(df_tracks_reco_scaled, exemplar_scaled, metric='l2')

cos_dist = dist(df_tracks_reco_scaled, exemplar_scaled, metric='cosine')

print(f'L1 dist is: {l1_dist.mean()}')
print(f'L2 dist is: {l2_dist.mean()}')
print(f'cos dist is: {cos_dist.mean()}')

L1 dist is: 3.344971715020311
L2 dist is: 1.442995943261194
cos dist is: 0.38254323319340566


### Model Evaluation (Comparison with Spotify's recommendation)

In this section, we will assess the recommendations provided by Spotify by examining their similarity to a set of predetermined input songs. The evaluation will employ various distance measures to quantify the likeness between the recommended tracks and the input tracks, thus providing a comprehensive analysis of the recommendation system's performance

### Volunteer 1

In [ ]:
# Volunteer 1
spotify_reco_songs_1 = ['4e3ZNTAV6PCrdYMUrUlMpQ', #electric touch
                      '4zmKGsrXjLmljb5fTaBTot', #snow on the beach
                      '7lLzAoAmxEKKAIovgzGP2D', #minaminamahal
                      '4XSIv5hJzP6kEQqsLli1eE', #kisame
                      '3eabrKx1t8T4sHLEAMmOXK', #euphoria smyang
                      '1K1KcmJgKaeqnsisQnhyE1', #mrs hollywood
                      '1HZR7A1UE9vBu24j9bZNN7', #future red velvet
                      '6ZUSwmLViVASwgHkYWyc3Z', #golden hour sb19
                      '3JKv5rt1H6o3TomGYdSsai', #grown ups sondia
                      '4EmH2iRucAgCOnhuJRotUi' #drifting tiesto
                     ]

In [ ]:
# Fetch song and artist names
track_details = sp.tracks(spotify_reco_songs_1)
songs_with_artists = [
    (track["name"], ", ".join([artist["name"] for artist in track["artists"]]))
    for track in track_details["tracks"]
]

# Print song names with artist names
print("Here are the recommended songs including its artists:")
for name, artists in songs_with_artists:
    print(f"{name} by {artists}")

Here are the recommended songs including its artists:
Electric Touch (feat. Fall Out Boy) (Taylor’s Version) (From The Vault) by Taylor Swift, Fall Out Boy
Snow On The Beach (feat. More Lana Del Rey) by Taylor Swift, Lana Del Rey
Mina(mahal ko) by rhodessa
Kisame by rhodessa
Euphoria by Smyang Piano
Mrs. Hollywood by Go-Jo
미래 by Red Velvet
golden hour - SB19 Remix by JVKE, SB19
Grown Ups by Sondia
Drifting by Tiësto


In [ ]:
spotify_reco_songs_with_audiofeat = get_audio_features(spotify_reco_songs_1)
spotify_reco_songs_with_audiofeat

,song_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,4e3ZNTAV6PCrdYMUrUlMpQ,0.547,0.8550,-4.700,0.0597,0.0164,0.000000,0.1440,0.2820,131.073,266467
1,4zmKGsrXjLmljb5fTaBTot,0.664,0.4010,-10.934,0.0344,0.8060,0.000003,0.1080,0.3150,109.846,229943
2,7lLzAoAmxEKKAIovgzGP2D,0.523,0.5650,-8.510,0.0636,0.4550,0.009520,0.1460,0.4970,165.869,282831
3,4XSIv5hJzP6kEQqsLli1eE,0.578,0.3050,-9.798,0.0258,0.9540,0.057800,0.2300,0.1970,99.914,212608
4,3eabrKx1t8T4sHLEAMmOXK,0.428,0.0985,-25.133,0.0446,0.9740,0.949000,0.1190,0.1530,70.003,203279
5,1K1KcmJgKaeqnsisQnhyE1,0.758,0.6550,-7.226,0.0305,0.1430,0.000000,0.1580,0.9720,128.019,143841
6,1HZR7A1UE9vBu24j9bZNN7,0.718,0.8650,-3.083,0.0413,0.4800,0.000000,0.1470,0.5600,100.013,215880
7,6ZUSwmLViVASwgHkYWyc3Z,0.463,0.6500,-4.584,0.0451,0.7140,0.001400,0.2310,0.3100,188.837,155996
8,3JKv5rt1H6o3TomGYdSsai,0.447,0.3200,-7.159,0.0356,0.7770,0.002650,0.0968,0.0989,119.829,259887
9,4EmH2iRucAgCOnhuJRotUi,0.625,0.9510,-4.866,0.0393,0.0718,0.585000,0.1640,0.3760,130.001,147692


In [ ]:
spotify_reco_songs_with_audiofeat1 = spotify_reco_songs_with_audiofeat.set_index(keys=['song_id'], drop=True).drop(columns=['duration_ms'])
spotify_reco_songs_with_audiofeat1

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
song_id,,,,,,,,,
4e3ZNTAV6PCrdYMUrUlMpQ,0.547,0.8550,-4.700,0.0597,0.0164,0.000000,0.1440,0.2820,131.073
4zmKGsrXjLmljb5fTaBTot,0.664,0.4010,-10.934,0.0344,0.8060,0.000003,0.1080,0.3150,109.846
7lLzAoAmxEKKAIovgzGP2D,0.523,0.5650,-8.510,0.0636,0.4550,0.009520,0.1460,0.4970,165.869
4XSIv5hJzP6kEQqsLli1eE,0.578,0.3050,-9.798,0.0258,0.9540,0.057800,0.2300,0.1970,99.914
3eabrKx1t8T4sHLEAMmOXK,0.428,0.0985,-25.133,0.0446,0.9740,0.949000,0.1190,0.1530,70.003
1K1KcmJgKaeqnsisQnhyE1,0.758,0.6550,-7.226,0.0305,0.1430,0.000000,0.1580,0.9720,128.019
1HZR7A1UE9vBu24j9bZNN7,0.718,0.8650,-3.083,0.0413,0.4800,0.000000,0.1470,0.5600,100.013
6ZUSwmLViVASwgHkYWyc3Z,0.463,0.6500,-4.584,0.0451,0.7140,0.001400,0.2310,0.3100,188.837
3JKv5rt1H6o3TomGYdSsai,0.447,0.3200,-7.159,0.0356,0.7770,0.002650,0.0968,0.0989,119.829


In [ ]:
scaler = MinMaxScaler()
spotify_reco_songs_with_audiofeat_scaled = scaler.fit_transform(spotify_reco_songs_with_audiofeat1)
exemplar_scaled = scaler.transform(exemplar)

In [ ]:
# Compute Pairwise Distances for 3 types of Similarity Measure
n = len(reco_songs)
l1_dist = dist(spotify_reco_songs_with_audiofeat_scaled,
               exemplar_scaled,
               metric='l1')

l2_dist = dist(spotify_reco_songs_with_audiofeat_scaled,
               exemplar_scaled,
               metric='l2')

cos_dist = dist(spotify_reco_songs_with_audiofeat_scaled,
                exemplar_scaled,
                metric='cosine')

print(f'L1 dist is: {l1_dist.mean()}')
print(f'L2 dist is: {l2_dist.mean()}')
print(f'cos dist is: {cos_dist.mean()}')

L1 dist is: 3.6442365622657937
L2 dist is: 1.6490037310126147
cos dist is: 0.38762757561584926


#### Comparing using Mahalanobis Distance

In [ ]:
# Combine all data into a DataFrame for scaling and covariance calculation
all_songs_df = pd.concat([reco_songs,
                          spotify_reco_songs_with_audiofeat1,
                          exemplar])
all_songs_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
song_id,,,,,,,,,
0BCPKOYdS2jbQ8iyB56Zns,0.577,0.7490,-7.215,0.0279,0.59900,0.011500,0.1830,0.2550,130.970
57RA3JGafJm5zRtKJiKPIm,0.682,0.6830,-6.444,0.0287,0.15600,0.000023,0.2730,0.6400,120.023
4LRPiXqCikLlN15c3yImP7,0.520,0.7310,-5.338,0.0557,0.34200,0.001010,0.3110,0.6620,173.930
23khhseCLQqVMCIT1WMAns,0.738,0.4820,-8.917,0.0402,0.02660,0.621000,0.1060,0.3520,93.457
2GiJYvgVaD2HtM8GqD9EgQ,0.611,0.7970,-7.627,0.0533,0.00543,0.001370,0.2600,0.5180,120.041
2QjOHCTQ1Jl3zawyYOpxh6,0.612,0.8070,-2.810,0.0336,0.04950,0.017700,0.1010,0.3980,124.053
7D0RhFcb3CrfPuTJ0obrod,0.371,0.2680,-10.506,0.0281,0.74800,0.051700,0.1040,0.1650,102.617
75JFxkI2RXiU7L9VXzMkle,0.557,0.4420,-7.224,0.0243,0.73100,0.000015,0.1100,0.2130,146.277
1qEmFfgcLObUfQm0j1W2CK,0.714,0.7280,-4.595,0.0468,0.29800,0.000000,0.1060,0.9010,114.996


In [ ]:
# Select features for Mahalanobis distance calculation
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
            'instrumentalness', 'liveness', 'valence', 'tempo']

# Scale the features
scaler = MinMaxScaler()
all_songs_scaled = scaler.fit_transform(all_songs_df[features])

In [ ]:
# Compute the covariance matrix and its inverse
cov_matrix = np.cov(all_songs_scaled, rowvar=False)
inv_cov_matrix = inv(cov_matrix)

In [ ]:
# Define a function to compute Mahalanobis distance between observations
def compute_mahalanobis(set1, set2, inv_cov):
    """Compute the Mahalanobis distance between two sets of points."""
    distances = np.zeros((set1.shape[0], set2.shape[0]))
    for i in range(set1.shape[0]):
        for j in range(set2.shape[0]):
            distances[i, j] = mahalanobis(set1[i], set2[j], inv_cov)
    return distances

In [ ]:
# Separate the scaled data back into the original groups
reco_songs_scaled = all_songs_scaled[:len(reco_songs)]
spotify_reco_songs_scaled = all_songs_scaled[len(reco_songs):-len(exemplar)]
exemplar_scaled = all_songs_scaled[-len(exemplar):]

In [ ]:
# Compute Mahalanobis distances as needed
distances_model_spotify = compute_mahalanobis(reco_songs_scaled,
                                              spotify_reco_songs_scaled,
                                              inv_cov_matrix)
distances_model_input = compute_mahalanobis(reco_songs_scaled,
                                            exemplar_scaled,
                                            inv_cov_matrix)
distances_spotify_input = compute_mahalanobis(spotify_reco_songs_scaled,
                                              exemplar_scaled,
                                              inv_cov_matrix)

# Aggregate the distances for analysis (example: average distance)
avg_distance_model_spotify = np.mean(distances_model_spotify)
avg_distance_model_input = np.mean(distances_model_input)
avg_distance_spotify_input = np.mean(distances_spotify_input)

# Output the aggregated distances for interpretation
avg_distance_model_spotify, avg_distance_model_input, avg_distance_spotify_input

print(f"MahDist between model and Spotify: {avg_distance_model_spotify}")
print(f"MahDist between model and input: {avg_distance_model_input}")
print(f"MahDist between Spotify and input: {avg_distance_spotify_input}")

MahDist between model and Spotify: 4.086570595912392
MahDist between model and input: 4.121935064270221
MahDist between Spotify and input: 4.512125877124747


### Volunteer 2

In [ ]:
# playlist 3
spotify_reco_songs_4 = ['4cxvludVmQxryrnx1m9FqL', #my shot
                      '0NJWhm3hUwIZSy5s0TGJ8q', #story of tonight
                      '4TTV7EcfroSLWzXRY6gLv6', #alexander hamilton
                      '6TBJkXHPhu3EsMk1bshwuI', #only leehi
                      '6HguG9HRb1Ke1bhihfE4m8', #sencerity is scary
                      '6dr7ekfhlbquvsVY8D7gyk', #aaron burr
                      '7EqpEBPOohgk7NnKvBGFWo', #wait for
                      '54Sc7mZQ1RM03STpk4SfaA', #helpless
                      '2G9lekfCh83S0lt2yfffBz', #farmer refuted
                      '3nJYcY9yvKP8Oi2Ml8brXt' #right hand man
                     ]

In [ ]:
# Fetch song and artist names
track_details = sp.tracks(spotify_reco_songs_4)
songs_with_artists = [
    (track["name"], ", ".join([artist["name"] for artist in track["artists"]]))
    for track in track_details["tracks"]
]

# Print song names with artist names
print("Here are the recommended songs including its artists:")
for name, artists in songs_with_artists:
    print(f"{name} by {artists}")

Here are the recommended songs including its artists:
My Shot by Lin-Manuel Miranda, Daveed Diggs, Okieriete Onaodowan, Leslie Odom Jr., Original Broadway Cast of Hamilton, Anthony Ramos
The Story of Tonight by Lin-Manuel Miranda, Okieriete Onaodowan, Daveed Diggs, Original Broadway Cast of Hamilton, Anthony Ramos
Alexander Hamilton by Leslie Odom Jr., Daveed Diggs, Okieriete Onaodowan, Lin-Manuel Miranda, Phillipa Soo, Christopher Jackson, Original Broadway Cast of Hamilton, Anthony Ramos
ONLY by LeeHi
Sincerity Is Scary by The 1975
Aaron Burr, Sir by Lin-Manuel Miranda, Leslie Odom Jr., Daveed Diggs, Okieriete Onaodowan, Anthony Ramos
Wait for It by Leslie Odom Jr., Original Broadway Cast of Hamilton
Helpless by Phillipa Soo, Original Broadway Cast of Hamilton
Farmer Refuted by Thayne Jasperson, Lin-Manuel Miranda, Original Broadway Cast of Hamilton
Right Hand Man by Lin-Manuel Miranda, Leslie Odom Jr., Original Broadway Cast of Hamilton


In [ ]:
spotify_reco_songs_with_audiofeat = get_audio_features(spotify_reco_songs_4)
spotify_reco_songs_with_audiofeat

,song_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,4cxvludVmQxryrnx1m9FqL,0.698,0.649,-6.764,0.4150,0.1500,0.000000,0.0903,0.568,180.466,333154
1,0NJWhm3hUwIZSy5s0TGJ8q,0.494,0.237,-10.579,0.0517,0.8350,0.000000,0.6480,0.625,95.219,91981
2,4TTV7EcfroSLWzXRY6gLv6,0.609,0.435,-7.861,0.2850,0.5220,0.000000,0.1180,0.552,131.995,236738
3,6TBJkXHPhu3EsMk1bshwuI,0.536,0.296,-7.451,0.0346,0.8920,0.000000,0.0873,0.151,122.907,240907
4,6HguG9HRb1Ke1bhihfE4m8,0.664,0.524,-5.178,0.0283,0.0495,0.000008,0.0750,0.475,91.027,225956
5,6dr7ekfhlbquvsVY8D7gyk,0.646,0.439,-10.837,0.8180,0.2720,0.000000,0.0945,0.528,160.387,156631
6,7EqpEBPOohgk7NnKvBGFWo,0.543,0.473,-9.638,0.1570,0.1240,0.000004,0.0922,0.519,86.534,193750
7,54Sc7mZQ1RM03STpk4SfaA,0.688,0.491,-7.494,0.2690,0.3340,0.000000,0.0914,0.737,79.009,249771
8,2G9lekfCh83S0lt2yfffBz,0.795,0.409,-8.334,0.3720,0.6840,0.000000,0.0548,0.577,77.879,112742
9,3nJYcY9yvKP8Oi2Ml8brXt,0.680,0.413,-9.410,0.4920,0.1770,0.000000,0.2050,0.383,158.576,321697


In [ ]:
spotify_reco_songs_with_audiofeat1 = spotify_reco_songs_with_audiofeat.set_index(
    keys=['song_id'], drop=True
).drop(columns=['duration_ms'])
spotify_reco_songs_with_audiofeat1

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
song_id,,,,,,,,,
4cxvludVmQxryrnx1m9FqL,0.698,0.649,-6.764,0.4150,0.1500,0.000000,0.0903,0.568,180.466
0NJWhm3hUwIZSy5s0TGJ8q,0.494,0.237,-10.579,0.0517,0.8350,0.000000,0.6480,0.625,95.219
4TTV7EcfroSLWzXRY6gLv6,0.609,0.435,-7.861,0.2850,0.5220,0.000000,0.1180,0.552,131.995
6TBJkXHPhu3EsMk1bshwuI,0.536,0.296,-7.451,0.0346,0.8920,0.000000,0.0873,0.151,122.907
6HguG9HRb1Ke1bhihfE4m8,0.664,0.524,-5.178,0.0283,0.0495,0.000008,0.0750,0.475,91.027
6dr7ekfhlbquvsVY8D7gyk,0.646,0.439,-10.837,0.8180,0.2720,0.000000,0.0945,0.528,160.387
7EqpEBPOohgk7NnKvBGFWo,0.543,0.473,-9.638,0.1570,0.1240,0.000004,0.0922,0.519,86.534
54Sc7mZQ1RM03STpk4SfaA,0.688,0.491,-7.494,0.2690,0.3340,0.000000,0.0914,0.737,79.009
2G9lekfCh83S0lt2yfffBz,0.795,0.409,-8.334,0.3720,0.6840,0.000000,0.0548,0.577,77.879


In [ ]:
scaler = MinMaxScaler()
spotify_reco_songs_with_audiofeat_scaled = scaler.fit_transform(spotify_reco_songs_with_audiofeat1)
exemplar_scaled = scaler.transform(exemplar)

In [ ]:
# Compute Pairwise Distances for 3 types of Similarity Measure
n = len(reco_songs)
l1_dist = dist(spotify_reco_songs_with_audiofeat_scaled,
               exemplar_scaled,
               metric='l1')

l2_dist = dist(spotify_reco_songs_with_audiofeat_scaled,
               exemplar_scaled,
               metric='l2')

cos_dist = dist(spotify_reco_songs_with_audiofeat_scaled,
                exemplar_scaled,
                metric='cosine')

print(f'L1 dist is: {l1_dist.mean()}')
print(f'L2 dist is: {l2_dist.mean()}')
print(f'cos dist is: {cos_dist.mean()}')

L1 dist is: 183.07755787393265
L2 dist is: 179.93447771087804
cos dist is: 0.7369674458416299


#### Comparing using Mahalanobis Distance

In [ ]:
# Combine all data into a single DataFrame for scaling and covariance calculation
all_songs_df = pd.concat([reco_songs, spotify_reco_songs_with_audiofeat1, exemplar])
all_songs_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
song_id,,,,,,,,,
0BCPKOYdS2jbQ8iyB56Zns,0.577,0.749,-7.215,0.0279,0.59900,0.011500,0.1830,0.255,130.970
57RA3JGafJm5zRtKJiKPIm,0.682,0.683,-6.444,0.0287,0.15600,0.000023,0.2730,0.640,120.023
4LRPiXqCikLlN15c3yImP7,0.520,0.731,-5.338,0.0557,0.34200,0.001010,0.3110,0.662,173.930
23khhseCLQqVMCIT1WMAns,0.738,0.482,-8.917,0.0402,0.02660,0.621000,0.1060,0.352,93.457
2GiJYvgVaD2HtM8GqD9EgQ,0.611,0.797,-7.627,0.0533,0.00543,0.001370,0.2600,0.518,120.041
2QjOHCTQ1Jl3zawyYOpxh6,0.612,0.807,-2.810,0.0336,0.04950,0.017700,0.1010,0.398,124.053
7D0RhFcb3CrfPuTJ0obrod,0.371,0.268,-10.506,0.0281,0.74800,0.051700,0.1040,0.165,102.617
75JFxkI2RXiU7L9VXzMkle,0.557,0.442,-7.224,0.0243,0.73100,0.000015,0.1100,0.213,146.277
1qEmFfgcLObUfQm0j1W2CK,0.714,0.728,-4.595,0.0468,0.29800,0.000000,0.1060,0.901,114.996


In [ ]:
# Select features for Mahalanobis distance calculation
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
            'instrumentalness', 'liveness', 'valence', 'tempo']

# Scale the features
scaler = MinMaxScaler()
all_songs_scaled = scaler.fit_transform(all_songs_df[features])

In [ ]:
# Compute the covariance matrix and its inverse
cov_matrix = np.cov(all_songs_scaled, rowvar=False)
inv_cov_matrix = inv(cov_matrix)

In [ ]:
# Define a function to compute Mahalanobis distance between two sets of observations
def compute_mahalanobis(set1, set2, inv_cov):
    distances = np.zeros((set1.shape[0], set2.shape[0]))
    for i in range(set1.shape[0]):
        for j in range(set2.shape[0]):
            distances[i, j] = mahalanobis(set1[i], set2[j], inv_cov)
    return distances

In [ ]:
# Separate the scaled data back into the original groups
reco_songs_scaled = all_songs_scaled[:len(reco_songs)]
spotify_reco_songs_scaled = all_songs_scaled[len(reco_songs):-len(exemplar)]
exemplar_scaled = all_songs_scaled[-len(exemplar):]

In [ ]:
# Compute Mahalanobis distances as needed
distances_model_spotify = compute_mahalanobis(reco_songs_scaled, spotify_reco_songs_scaled, inv_cov_matrix)
distances_model_input = compute_mahalanobis(reco_songs_scaled, exemplar_scaled, inv_cov_matrix)
distances_spotify_input = compute_mahalanobis(spotify_reco_songs_scaled, exemplar_scaled, inv_cov_matrix)

# Aggregate the distances for analysis (example: average distance)
avg_distance_model_spotify = np.mean(distances_model_spotify)
avg_distance_model_input = np.mean(distances_model_input)
avg_distance_spotify_input = np.mean(distances_spotify_input)

# Output the aggregated distances for interpretation
avg_distance_model_spotify, avg_distance_model_input, avg_distance_spotify_input

print(f"MahDist between model and Spotify: {avg_distance_model_spotify}")
print(f"MahDist between model and input: {avg_distance_model_input}")
print(f"MahDist between Spotify and input: {avg_distance_spotify_input}")

MahDist between model and Spotify: 4.1787705932707455
MahDist between model and input: 4.247212849364077
MahDist between Spotify and input: 4.330190857058009


## In summary: </h4>

| Metric                          | Model | Volunteer 1 | Volunteer 2 |
|---------------------------------|-----------|-------------|-------------|
| Manhattan Distance (L1 Distance) | 3.34      | 3.64        | 183.07      |
| Euclidean Distance (L2 Distance) | 1.44      | 1.65        | 179.93      |
| Cosine Distances                | 0.38      | 0.39       | 0.73        |
| Mahalanobis Distance            | 4.24/4.12     | 4.51        | 4.33        |